In [ ]:
# ==========================================
# Cell 1: 全局超参数与路径配置区 (所有可调参数均在此处)
# ==========================================

# 1. 路径配置 (请根据实际情况修改基础路径)
DATA_BASE_DIR = "./data"          # 存放所有抽帧图片的根目录
CSV_BASE_DIR = "./annotations"    # 存放所有CSV标注文件的根目录

# 2. 训练超参数
BATCH_SIZE = 32                   # 批次大小 (如果显存不够，可调小至 16 或 8)
EPOCHS = 20                       # 训练总轮数
LEARNING_RATE = 1e-4              # 初始学习率
WEIGHT_DECAY = 1e-4               # 权重衰减 (防止过拟合)
EARLY_STOPPING_PATIENCE = 5       # 早停耐心值 (连续5个epoch验证集准确率未提升则停止)

# 3. 图像尺寸与模型参数
IMAGE_SIZE = 224                  # ViT的标准输入分辨率 (请勿随意修改，除非使用384的预训练权重)
MODEL_NAME = "vit_base_patch16_224" # timm库中的ViT模型名称
NUM_CLASSES = 9                   # 景别类别总数 (0到8，共9类)

# 4. 景别类别映射字典 (根据 README.txt 设定)
# 注意：代码逻辑中会自动过滤掉 9 (Not available) 的数据
CLASS_MAPPING = {
    0: 'FS (Foreground shot)',
    1: 'ECU (Extreme close up)',
    2: 'CU (Close up)',
    3: 'MCU (Medium close up)',
    4: 'MS (Medium shot)',
    5: 'MLS (Medium long shot)',
    6: 'LS (Long shot)',
    7: 'ELS (Extreme long shot)',
    8: 'INS (Insert)'
}

# 5. 硬件与随机种子配置
SEED = 42                         # 固定随机种子，保证实验可复现
VAL_SPLIT_RATIO = 0.2             # 验证集划分比例 (20%用于验证)

In [ ]:
# ==========================================
# Cell 2: 导入所有依赖库并配置硬件环境
# ==========================================
import os
import glob
import torch
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import timm
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import random

# 固定随机种子
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

# 检测并配置设备 (优先使用 GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🎬 当前训练设备: {device}")

In [ ]:
# ==========================================
# Cell 3: 自定义景别数据集读取逻辑
# ==========================================

class ShotScaleDataset(Dataset):
    def __init__(self, csv_dir, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.samples = []
        
        # 遍历所有CSV文件
        csv_files = glob.glob(os.path.join(csv_dir, "*.csv"))
        for csv_path in csv_files:
            filename = os.path.basename(csv_path)
            # 根据 README 规则解析目录名: {year}_{director}_-_{title}.csv
            # 例如: 1950_antonioni_-_cronaca_di_un_amore.csv
            movie_folder_name = filename.replace(".csv", "")
            director = movie_folder_name.split("_-_")[0].split("_", 1)[1]
            
            # 构建该电影图片的存放路径
            img_folder_path = os.path.join(data_dir, director, movie_folder_name)
            
            # 读取CSV
            df = pd.read_csv(csv_path)
            
            # 过滤掉 shotscale == 9 (NA) 的行
            df = df[df['shotscale'] != 9]
            
            for index, row in df.iterrows():
                # 根据CSV的行索引(其实就是第一列 Unnamed: 0) 计算图片名
                # 假设索引 0 对应 00001.jpg
                img_idx = int(row.iloc[0]) + 1
                img_name = f"{img_idx:05d}.jpg"
                img_path = os.path.join(img_folder_path, img_name)
                
                label = int(row['shotscale'])
                
                # 将有效路径和标签存入列表
                self.samples.append((img_path, label))
                
        print(f"✅ 成功加载数据集，共找到 {len(self.samples)} 帧有效训练图像。")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        try:
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            print(f"⚠️ 读取图片失败: {img_path}, 错误: {e}")
            # 如果出错，返回一张纯黑图作为替代，避免训练中断
            image = Image.new('RGB', (IMAGE_SIZE, IMAGE_SIZE), (0, 0, 0))
            
        if self.transform:
            image = self.transform(image)
            
        return image, label

# ------------------------------------------
# 定义 Transformer 专属的数据预处理
# ⚠️ 坚决不使用 RandomCrop，严格保护影视原始构图比例！
# ------------------------------------------
data_transforms = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)), # 直接拉伸或者按比例缩放填充以匹配输入
    transforms.RandomHorizontalFlip(p=0.5),      # 左右镜像不影响景别判断
    transforms.ColorJitter(brightness=0.2, contrast=0.2), # 模拟不同光照
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # ImageNet标准归一化
])

val_transforms = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
# ==========================================
# Cell 4: 实例化 Dataset 并划分为训练集和验证集
# ==========================================

# 初始化完整数据集
full_dataset = ShotScaleDataset(csv_dir=CSV_BASE_DIR, data_dir=DATA_BASE_DIR, transform=data_transforms)

# 计算划分长度
val_size = int(len(full_dataset) * VAL_SPLIT_RATIO)
train_size = len(full_dataset) - val_size

# 随机切分
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# 验证集需要使用纯净的 val_transforms (去除数据增强)
val_dataset.dataset.transform = val_transforms

# 创建 DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)

print(f"🎞️ 训练集图像数: {train_size} | 验证集图像数: {val_size}")

In [ ]:
# ==========================================
# Cell 5: 构建 Vision Transformer 并替换分类头
# ==========================================

# 使用 timm 直接加载预训练的 ViT 模型，并自动将输出类别数修改为我们的 NUM_CLASSES
model = timm.create_model(MODEL_NAME, pretrained=True, num_classes=NUM_CLASSES)

# 将模型搬运到 GPU (或 CPU)
model = model.to(device)

print(f"🚀 模型 {MODEL_NAME} 加载完毕，输出层已调整为 {NUM_CLASSES} 类。")

In [ ]:
# ==========================================
# Cell 6: 定义损失函数、优化器与学习率衰减策略
# ==========================================

# 交叉熵损失函数
criterion = nn.CrossEntropyLoss()

# Transformer 的标配：AdamW 优化器
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

# 余弦退火学习率调度器 (让学习率平滑下降)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

In [ ]:
# ==========================================
# Cell 7: 训练主循环 (包含早停与最优权重保存机制)
# ==========================================

best_val_acc = 0.0
epochs_no_improve = 0
best_model_path = 'best_shotscale_vit.pth'

for epoch in range(EPOCHS):
    print(f"\n--- Epoch {epoch+1}/{EPOCHS} ---")
    
    # ---------------- 训练阶段 ----------------
    model.train()
    train_loss = 0.0
    train_correct = 0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        train_correct += torch.sum(preds == labels.data)
        
    scheduler.step() # 更新学习率
    
    epoch_train_loss = train_loss / train_size
    epoch_train_acc = train_correct.double() / train_size
    
    # ---------------- 验证阶段 ----------------
    model.eval()
    val_loss = 0.0
    val_correct = 0
    
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            val_correct += torch.sum(preds == labels.data)
            
    epoch_val_loss = val_loss / val_size
    epoch_val_acc = val_correct.double() / val_size
    
    print(f"Train Loss: {epoch_train_loss:.4f} | Train Acc: {epoch_train_acc:.4f}")
    print(f"Val Loss:   {epoch_val_loss:.4f} | Val Acc:   {epoch_val_acc:.4f}")
    
    # ---------------- 早停与保存最优权重 ----------------
    if epoch_val_acc > best_val_acc:
        best_val_acc = epoch_val_acc
        epochs_no_improve = 0
        torch.save(model.state_dict(), best_model_path)
        print(f"🌟 验证集准确率创新高! 权重已保存至 {best_model_path}")
    else:
        epochs_no_improve += 1
        print(f"⚠️ 验证集准确率未提升 (累计 {epochs_no_improve} 个 Epoch)")
        
    if epochs_no_improve >= EARLY_STOPPING_PATIENCE:
        print(f"🛑 触发早停机制，训练结束！")
        break

In [ ]:
# ==========================================
# Cell 8: 加载最优权重，输出评估报告与混淆矩阵
# ==========================================

# 加载最好的模型
model.load_state_dict(torch.load(best_model_path))
model.eval()

all_preds = []
all_labels = []

print("正在生成最终评估报告，请稍候...")
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# 获取真实存在的类别名称用于绘图
target_names = [CLASS_MAPPING[i] for i in range(NUM_CLASSES) if i in set(all_labels)]

# 打印分类报告
print("\n📝 分类报告 (Classification Report):")
print(classification_report(all_labels, all_preds, target_names=target_names))

# 绘制混淆矩阵
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=target_names, yticklabels=target_names)
plt.title('Shot Scale Classification Confusion Matrix')
plt.ylabel('True Shot Scale')
plt.xlabel('Predicted Shot Scale')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()